# Exploratory Visualization

_run below cell once_

In [11]:
import chart_studio.plotly as py
import pandas as pd
import numpy as np
import json, requests
from pandas.io.json import json_normalize
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

query = {}

results = requests.post('https://www.lobbyview.org/public/api/reports', data = json.dumps(query), verify=False)

json_object = results.json() 

with open('data_file.json', 'w') as outfile:
    json.dump(json_object, outfile)

/Users/hopetambala/Documents/development/school/LobbyView/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [12]:
with open('data_file.json', 'r') as f:
    data = json.load(f)

## Parse Sponsors and Lobbyists from JSON Object

In [13]:
for result in data['result']:
#     print(result)
    if 'specific_issues' in list(result.keys()):
        for issue in result['specific_issues']:
            if 'bills_by_algo' in list(issue.keys()):
                for sponsors in issue['bills_by_algo']:
                    lobbyists = []
                    spons = []
                    
                    lastname = sponsors['sponsor']['lastname']
                    firstname = sponsors['sponsor']['firstname']
                    
                    name = lastname + ", " + firstname
                    
                    party = sponsors['sponsor']['party']
                    
                    spons.append([name, party])
                    
                    for lobbyist in issue['lobbyists']:
                        lobbyists.append(lobbyist)

                    result["lobbyists"] = lobbyists
                    result["sponsors"] = spons
                    
            else:
                continue
    else:
        continue
                    

KeyError: 'lobbyists'

In [14]:
df = json_normalize(data['result'])

In [15]:
print(df.columns)
print(df.info())
df.describe()

Index(['issue_codes', 'received', 'has_amendments', 'client_name', 'period',
       'registrant', 'amount', 'specific_issues', 'year',
       'is_latest_amendment', 'type', 'id', 'alternate_name',
       'client.legal_name', 'client.name', 'client.naics', 'client.gvkey',
       'client.ticker', 'client.id', 'client.bvdid', 'lobbyists', 'sponsors',
       'affiliated_orgs', 'foreign_entities'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
issue_codes            5000 non-null object
received               5000 non-null int64
has_amendments         5000 non-null bool
client_name            4725 non-null object
period                 5000 non-null object
registrant             5000 non-null object
amount                 3533 non-null float64
specific_issues        4692 non-null object
year                   5000 non-null int64
is_latest_amendment    5000 non-null bool
type                   5000 non-null ob

,received,amount,year
count,5.000000e+03,3.533000e+03,5000.000000
mean,1.268565e+09,8.967131e+04,2009.433600
std,1.574466e+08,3.467687e+05,5.066629
min,9.183636e+08,0.000000e+00,1999.000000
25%,1.155269e+09,1.950000e+04,2006.000000
50%,1.278572e+09,3.000000e+04,2010.000000
75%,1.397690e+09,6.000000e+04,2014.000000
max,1.526436e+09,1.090000e+07,2018.000000


#### Only get relevant columns

In [16]:
columns=['issue_codes','client_name','period','registrant','amount','specific_issues','year','type','id','alternate_name', 'client.legal_name','client.name','lobbyists','sponsors']


In [17]:
df[columns]

,issue_codes,client_name,period,registrant,amount,specific_issues,year,type,id,alternate_name,client.legal_name,client.name,lobbyists,sponsors
0,[BUD],TRANSPORTATION DISTRICT COMMISSION OF HAMPTON ...,Mid-Year (Jan 1 - Jun 30),HOLLAND & KNIGHT LLP,20000.0,"[{'text': '', 'gov_entities': ['Federal Transi...",2001,MID-YEAR REPORT,0CA354DE-D9EF-4D44-ADF3-7C50F22A9174,TRANSPORTATION DISTRICT COMMISSION OF HAMPTON ...,TRANSPORTATION DISTRICT COMMISSION OF HAMPTON ...,TRANSPORTATION DISTRICT COMMISSION OF HAMPTON ...,NaN,NaN
1,[HCR],NYU HOSPITALS CENTER,1st Quarter (Jan 1 - Mar 31),NYU Hospitals Center,20000.0,"[{'text': 'AHCA,Continued funding and insuranc...",2017,FIRST QUARTER REPORT,0CA35FEE-25F5-457B-BB35-51751AD9A20A,NYU HOSPITALS CENTER,NYU HOSPITALS CENTER,NYU HOSPITALS CENTER,"[Ventresca-Ecroyd, Gilda]","[[Klobuchar, Amy, Democrat]]"
2,[DEF],TRUETECH INC,3rd Quarter (July 1 - Sep 30),PR/STRATEGIES INTERNATIONAL LLC,5000.0,"[{'text': 'military supply issues ', 'gov_ent...",2010,THIRD QUARTER REPORT,0CA37843-506D-4673-AD95-BED34BB66604,TRUETECH INC,TRUETECH INC,TRUETECH INC,NaN,NaN
3,[DEF],ENVISION CORP,2nd Quarter (Apr 1 - June 30),"DPR Consulting, LLC",30000.0,[{'text': 'FY 2009 Defense Appropriations Bill...,2008,SECOND QUARTER REPORT,0CA382C8-1DF9-4F92-8C35-408B5E77934A,ENVISION CORPORATION,ENVISION CORP,ENVISION CORPORATION,NaN,NaN
4,[],WESTERN MICHIGAN UNIVERSITY,Year-End (July 1 - Dec 31),WESTERN MICHIGAN UNIVERSITY,89625.0,NaN,2004,YEAR-END REPORT,0CA38934-DEED-4F39-86C3-9FC7ADCE5E3F,WESTERN MICHIGAN UNIVERSITY,WESTERN MICHIGAN UNIVERSITY,WESTERN MICHIGAN UNIVERSITY,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,[IND],EASTERN BAND OF CHEROKEE INDIANS,4th Quarter (Oct 1 - Dec 31),"Ietan Consulting, LLC",45000.0,"[{'text': 'Appropriations, Federal Recognition...",2012,FOURTH QUARTER REPORT,17395418-BC90-44D6-A849-F6B9193E557B,EASTERN BAND OF CHEROKEE INDIANS,EASTERN BAND OF CHEROKEE INDIANS,EASTERN BAND OF CHEROKEE INDIANS,NaN,NaN
4996,"[ECN, HOU, RES, BAN, SMB, BUD]",None,UNDETERMINED,Theodosiou Consultants,NaN,"[{'text': '', 'issue': 'BAN', 'id': 'HQi9XKG4o...",2005,REGISTRATION,1739598E-E0B4-44BF-B504-2A5F1E643EE7,GAHANIA REGINA,None,GAHANIA REGINA,NaN,NaN
4997,"[ECN, MED, ENG]",BIOMEDICAL RESEARCH FOUNDATION,3rd Quarter (July 1 - Sep 30),Charlie McBride Associates,7200.0,[{'text': 'To monitor legislation affecting me...,2010,THIRD QUARTER REPORT,17397D1B-5886-400C-8846-3A22F378F543,Biomedical Research Foundation of Northwest Lo...,BIOMEDICAL RESEARCH FOUNDATION,Biomedical Research Foundation of Northwest Lo...,NaN,NaN
4998,"[HCR, SCI, FOR, ECN, TEC, IMM, TRD, EDU, AGR]",AFRICANFEDERAL GROUP LLC,UNDETERMINED,"African Federal Group, LLC",NaN,[{'text': 'African Development issues. African...,2012,REGISTRATION AMENDMENT,173985A3-3532-432F-93FF-262F1F7E9E67,AFRICANFEDERAL GROUP LLC,AFRICANFEDERAL GROUP LLC,AFRICANFEDERAL GROUP LLC,NaN,NaN


#### Remove rows without data regarding lobbyists and sponsors

In [18]:
df['lobbyists'].replace('', np.nan, inplace=True)
df['sponsors'].replace('', np.nan, inplace=True)

In [19]:
df.dropna(subset=['lobbyists'], inplace=True)
df.dropna(subset=['sponsors'], inplace=True)

In [20]:
df.head()

,issue_codes,received,has_amendments,client_name,period,registrant,amount,specific_issues,year,is_latest_amendment,...,client.name,client.naics,client.gvkey,client.ticker,client.id,client.bvdid,lobbyists,sponsors,affiliated_orgs,foreign_entities
1,[HCR],1492187423,False,NYU HOSPITALS CENTER,1st Quarter (Jan 1 - Mar 31),NYU Hospitals Center,20000.0,"[{'text': 'AHCA,Continued funding and insuranc...",2017,False,...,NYU HOSPITALS CENTER,None,None,Unlisted,None,US127875960L,"[Ventresca-Ecroyd, Gilda]","[[Klobuchar, Amy, Democrat]]",NaN,NaN
5,"[HCR, LBR, TAX, CIV]",1295553772,False,INDEPENDENT ELECTRICAL CONTRACTORS INC,4th Quarter (Oct 1 - Dec 31),"Independent Electrical Contractors, Inc.",54513.0,[{'text': 'Employee Free Choice Act - H.R. 140...,2010,False,...,"INDEPENDENT ELECTRICAL CONTRACTORS, INC",236115,None,Unlisted,None,US148595324L,"[Hinton, Steve, Worth, Brian]","[[Frank, Barney, Democrat]]",NaN,NaN
6,"[TAX, TRD, GAM, BAN, CPT, CSP]",1437057607,False,MasterCard Inc,2nd Quarter (Apr 1 - June 30),The Smith-Free Group,70000.0,"[{'text': 'H.R.9, Innovation Act (All sections...",2015,False,...,MASTERCARD Worldwide,522210,160225,Unlisted,None,CL590496804,"[Barbour, Andrew, Deuser, Jack S., Locke, W. T...","[[Carper, Thomas, Democrat]]",NaN,NaN
14,[TRD],1248117294,False,MILLIKEN & CO,2nd Quarter (Apr 1 - June 30),MILLIKEN & COMPANY,60000.0,[{'text': 'HR 2847; Defense Approps (Yet to be...,2009,False,...,MILLIKEN & COMPANY,339999,None,Unlisted,None,US120565447L,"[Dutilh, Katherine, Nash, John, Tantillo, Augu...","[[Leahy, Patrick, Democrat]]",NaN,NaN
15,[LBR],1240263069,False,INTERNATIONAL UNION OF BRICKLAYERS & ALLIED CRAFT,1st Quarter (Jan 1 - Mar 31),Bartley O'Hara,NaN,[{'text': 'Working to gather support for prope...,2009,False,...,International Union of Bricklayers & Allied Cr...,238140,None,Unlisted,None,US130397937L,"[O'Hara, Bartley]","[[Lieberman, Joseph, Independent]]",NaN,NaN


#### Create CSV with Sponsors and Lobbyists Data for reference

In [ ]:
df.to_csv('../data_aggregation/LobbyViewDataWithSponsors+Lobbyists'+'.csv')

## Exploratory Data visualization on numerical features

_get plotly working offline_

In [21]:
dfviz = df

In [32]:
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [33]:
dfviz['client.legal_name'].iplot(kind='hist', xTitle='Clients',
                  yTitle='count', title='Count of of Clients')

In [34]:
dfviz['registrant'].iplot(kind='hist', xTitle='Registrants',
                  yTitle='count', title='Number of Times a Firm Has Registered a Report')

In [38]:
dfviz.iplot(
    x='year',
    y='amount',
    # Specify the category
    categories='period',
    xTitle='Year',
    yTitle='Amount Given',
    title='Amount Given Compared to year')

# Amount of Money Lobbied For Per Year

In [39]:
data1 = [go.Bar(x=dfviz.year,
            y=dfviz.amount)]

py.iplot(data1, filename='lobbydata1', width=800, height=400)

In [40]:
dfviz['registrant_freq'] = dfviz.groupby('registrant')['registrant'].transform('count')
df_drop_less_registrants = dfviz[~(dfviz['registrant_freq'] <= 1)]  


df_reg_amounts = pd.DataFrame({'amount_sum' : df_drop_less_registrants.groupby( ["registrant", "registrant_freq","year"] )['amount'].sum()}).reset_index()
df_reg_amounts.head()


,registrant,registrant_freq,year,amount_sum
0,AKIN GUMP STRAUSS HAUER & FELD,4,2009,10000.0
1,AKIN GUMP STRAUSS HAUER & FELD,4,2012,0.0
2,AKIN GUMP STRAUSS HAUER & FELD,4,2016,100000.0
3,AKIN GUMP STRAUSS HAUER & FELD,4,2017,120000.0
4,"ALPINE GROUP, INC.",8,2006,60000.0


# Amount of Money Lobbied For Per Registrant (Colored by Amount of Times They've Lobbied)

In [41]:
data2 = [go.Scatter(
    y=df_reg_amounts.amount_sum,
    x=df_reg_amounts.year, 
    mode='markers',
    marker_color=df_reg_amounts['registrant_freq'],
    text=df_reg_amounts['registrant'])] # hover text goes here

py.iplot(data2, filename='lobbydata2', width=200, height=400)

# Amount of Money Lobbied For Per Registrant

In [42]:
df_sum_amounts = pd.DataFrame({'amount_sum' : dfviz.groupby( [ "registrant"] )['amount'].sum()}).reset_index()
df_sum_amounts = df_sum_amounts.sort_values(by=['amount_sum'], ascending=False)
df_sum_amounts.head()

,registrant,amount_sum
117,Chamber of Commerce of the U.S.A.,7890000.0
123,ConocoPhillips,4338992.0
243,LOCKHEED MARTIN CORPORATION,4030000.0
184,General Motors Company,3140000.0
250,MARATHON OIL CORPORATION,3030000.0


In [43]:
data3 = [go.Bar(y=df_sum_amounts.registrant,
            x=df_sum_amounts.amount_sum, orientation='h')]

py.iplot(data3, filename='lobbydata3', width=200, height=400)

## Amount of Money Lobbied For Per Registrant (more than 500,000 dollars)

In [44]:
df_sum_amounts_lessthan500 = df_sum_amounts[~(df_sum_amounts['amount_sum'] <= 500000)] 
data4 = [go.Bar(y=df_sum_amounts_lessthan500.registrant,
            x=df_sum_amounts_lessthan500.amount_sum, orientation='h')]

py.iplot(data4, filename='lobbydata4', width=200, height=400)